In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipfx.spike_detector as spkd
from ipfx.dataset.create import create_ephys_data_set
from ipfx.utilities import drop_failed_sweeps
from ipfx.epochs import get_stim_epoch
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from scipy.io import savemat

from helpers.features_100Hz_traces import extract_spike_features, get_cell_features

In [3]:
root_path = 'D:/mpinb/data/mouse_v1'
data_path = root_path + '/raw/ephys/'
save_path = root_path + '/processed/ephys/'

In [4]:
file_manifest = pd.read_csv(root_path + "/2021-09-13_mouse_file_manifest.csv")
metadata = pd.read_csv(root_path + "/20200711_patchseq_metadata_mouse.csv")
metadata = metadata.loc[metadata['dendrite_type']!='spiny', :]
met_types = pd.read_csv(root_path + "/visp_met_cell_assignments_text_names.csv")
print("Mouse GABAergic cells: %d (raw) %d (MET)" % (len(metadata), len(met_types)))

Mouse GABAergic cells: 4299 (raw) 495 (MET)


In [5]:
metadata = metadata.merge(met_types, how='outer', left_on='cell_specimen_id', right_on='specimen_id')
metadata = metadata.loc[metadata['met_type'].notna(), :]
metadata

,project,cell_specimen_id,cell_specimen_name,hemisphere,structure,donor_id,donor_name,biological_sex,age,ethnicity,...,neuron_reconstruction_type,cell_soma_normalized_depth,depth_from_pia_um,ephys_session_id,transcriptomics_sample_id,transcriptomics_batch,Tree_first_cl_label,Unnamed: 21,specimen_id,met_type
0,mIVSCC-MET,601506507,Vip-IRES-Cre;Ai14-331294.04.01.01,left,VISpm4,331294,Vip-IRES-Cre;Ai14-331294,M,P55,NaN,...,full,0.361015,NaN,601506492,PS0810_E1-50_S88,BT035,CS180626100018,Vip Gpc3 Slc18a3,601506507.0,Vip-MET-2
2,mIVSCC-MET,601803754,Sst-IRES-Cre;Ai14-331663.04.02.02,right,VISp2/3,331663,Sst-IRES-Cre;Ai14-331663,F,P57,NaN,...,full,0.164240,NaN,601803733,PS0817_E1-50_S25,BT036,CS180626100035,Sst Calb2 Pdlim5,601803754.0,Sst-MET-3
3,mIVSCC-MET,601808698,Sst-IRES-Cre;Ai14-331663.04.02.03,right,VISp5,331663,Sst-IRES-Cre;Ai14-331663,F,P57,NaN,...,full,0.483030,NaN,601808619,PS0817_E1-50_S26,BT036,CS180626100043,Sst Hpse Cbln4,601808698.0,Sst-MET-8
4,mIVSCC-MET,601810307,Sst-IRES-Cre;Ai14-331663.04.02.04,right,VISp2/3,331663,Sst-IRES-Cre;Ai14-331663,F,P57,NaN,...,full,0.295656,NaN,601810292,PS0817_E1-50_S27,BT036,CS180626100034,Sst Calb2 Necab1,601810307.0,Sst-MET-3
5,mIVSCC-MET,601811091,Sst-IRES-Cre;Ai14-331663.03.01.01,left,VISp6a,331663,Sst-IRES-Cre;Ai14-331663,F,P57,NaN,...,full,0.766595,NaN,601811076,PS0817_E1-50_S28,BT036,CS180626100046,Sst Esm1,601811091.0,Sst-MET-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4263,mIVSCC-MET,992268582,Vipr2-IRES2-Cre;Slc32a1-IRES2-FlpO;Ai65-499514...,right,VISp2/3,499514,Vipr2-IRES2-Cre;Slc32a1-IRES2-FlpO;Ai65-499514,M,P57,NaN,...,full,0.222557,NaN,992268560,SM-J39ZH_S506_E1-50,RSC-240,CS180626100060,Pvalb Vipr2,992268582.0,Pvalb-MET-5
4266,mIVSCC-MET,992386952,Sncg-IRES2-FlpO-neo;Ai65F-499189.04.02.02,right,VISl1,499189,Sncg-IRES2-FlpO-neo;Ai65F-499189,F,P60,NaN,...,full,0.143579,NaN,992386928,SM-J39ZH_S505_E1-50,RSC-240,CS180626100011,Sncg Vip Itih5,992386952.0,Sncg-MET-1
4277,mIVSCC-MET,992830261,Sncg-IRES2-FlpO-neo;Ai65F-499191.05.02.02,right,VISp5,499191,Sncg-IRES2-FlpO-neo;Ai65F-499191,F,P62,NaN,...,full,0.691884,NaN,992830126,SM-J39ZH_S566_E1-50,RSC-240,CS180626100008,Sncg Slc17a8,992830261.0,Sncg-MET-3
4279,mIVSCC-MET,993245688,Sncg-IRES2-FlpO-neo;Ai65F-500414.05.01.01,left,VISp2/3,500414,Sncg-IRES2-FlpO-neo;Ai65F-500414,M,P55,NaN,...,full,0.270298,NaN,993245664,SM-J39TJ_S275_E1-50,RSC-242,CS180626100010,Sncg Gpr50,993245688.0,Sncg-MET-1


In [6]:
# Set variables to save
all_ephys_features = pd.DataFrame()
V, specimen_id, structure, t_type, met_type = [], [], [], [], []

# Set stimulus duration (500 ms as in our data)
stim_duration = 0.5 

# Set time window before and after stimulus (100 ms as in our data)
t_no_stim = 0.1

# Define standardized number samples, make sure that sampling freq is above Nyquist (> 20kHz works)
t_standardized = np.arange(0, 0.7, 1/30000)

for cell_id in np.arange(len(metadata)):
    
    # Initialize variable to know if cell should be saved
    valid_cell = False
        
    # Get current specimen id
    specimen_id_now = metadata.iloc[cell_id, :]["cell_specimen_id"]
    
    # Get ephys file name
    nbw_idx = (file_manifest["file_type"]=="nwb") & (
        file_manifest["cell_specimen_id"]==float(specimen_id_now))
    nbw_file = file_manifest.loc[nbw_idx,:]["file_name"].values[0]

    # Load experiment for current specimen
    path_to_file = data_path + nbw_file.split("_")[0] + "/" + nbw_file
    data_set = create_ephys_data_set(nwb_file=path_to_file)
    
    # Check if current file is valid
    try:
        # Refine current data
        drop_failed_sweeps(data_set)
    except TypeError:
        print("%4.f TypeError" % cell_id)
        continue

    # get sweep table of Long Square sweeps
    long_square_table = data_set.filtered_sweep_table(
        stimuli=data_set.ontology.long_square_names
    )
    long_square_sweeps = data_set.sweep_set(long_square_table.sweep_number)
    
    # Initialize 100 Hz variable
    best_init_freq = np.inf

    for sweep_id in np.arange(len(long_square_sweeps.sweep_number)):

        # Compute variables in tVI format (s,mV,mA)
        t = long_square_sweeps.t[sweep_id]
        v = long_square_sweeps.v[sweep_id]
        i = long_square_sweeps.i[sweep_id]

        # Get current stim onset
        stim_onset_index, stim_offset_index = get_stim_epoch(i)
        t_stim_onset = t[stim_onset_index]
        t_stim_offset = t_stim_onset + stim_duration

        # Detect spikes
        putative_spikes = spkd.detect_putative_spikes(v, t, t_stim_onset, t_stim_offset)
        peaks = spkd.find_peak_indexes(v, t, putative_spikes, t_stim_offset)
        peak_indexes, spike_indexes = spkd.filter_putative_spikes(v, t, putative_spikes, peaks)

        # Check if it's a valid sweep (Freq>5Hz) and that V and I aren't missing
        if (len(spike_indexes) >= 2) & (len(v) == len(i)):

            # Calculate initial firing frequency in Hz
            init_freq = 1 / (t[spike_indexes[1]] - t[spike_indexes[0]])
            
            # Calculate the coefficient of variation of spike amplitudes
            trough_indexes = spkd.find_trough_indexes(v, t, spike_indexes, peak_indexes)
            spike_amp = v[spike_indexes] - v[trough_indexes.astype(int)]
            CV_amp = np.std(spike_amp) / np.mean(spike_amp)

            # Store the selected 100 Hz trace
            if np.abs(100-init_freq)<=np.abs(100-best_init_freq):
                
                if init_freq==best_init_freq:
                    if best_CV_amp >= CV_amp:
                        # update saved CV of spike amplitudes
                        best_CV_amp = CV_amp
                        # update saved sweep
                        best_sweep_id = sweep_id
                else:
                    # save initial firing rate
                    best_init_freq = init_freq
                    # save CV of spike amplitudes
                    best_CV_amp = CV_amp
                    # save corresponding sweep
                    best_sweep_id = sweep_id
                    
                # get sampling rate
                sampling_rate = long_square_sweeps.sampling_rate[sweep_id]
                # get trace from 0ms to 500ms after stimulus onset
                idx_start = int((t_stim_onset-t_no_stim)*sampling_rate)
                idx_mid1  = int((t_stim_onset+stim_duration)*sampling_rate)
                V_tmp1 = v[range(idx_start, idx_mid1)]
                # set number of samples after stimulus, add delta of 0.5ms to   
                # avoid unwanted fluctuations directly after end of stimulus
                V_tmp2 = v[int(stim_offset_index + 0.5/1000*sampling_rate):] 
                # smooth decay 
                V_tmp2 = V_tmp2[v[idx_mid1] > V_tmp2]
                # concatenate voltages (end of experiment determined during standardization)
                V_crop = np.concatenate((V_tmp1, V_tmp2[:int(2*t_no_stim*sampling_rate)]))
                
                if int(0.7*sampling_rate)<=len(V_crop):
                    # standardize voltage trace
                    interp = interp1d(np.arange(0, len(V_crop))/sampling_rate, V_crop)
                    best_sweep = interp(t_standardized)
                    curr_best_sweep = np.array([i[idx_mid1]])
                    # set cell as valid (note by definition all met cells are valid)
                    valid_cell = True

    if valid_cell:
        
        features_now = get_cell_features(specimen_id_now, t_standardized, curr_best_sweep, 
                                         np.expand_dims(best_sweep, axis=1), 
                                         start=t_no_stim, end=t_no_stim+stim_duration)
        features_now.to_csv(save_path + 'features/' + str(specimen_id_now) + '.csv') 
        all_ephys_features = pd.concat([all_ephys_features, features_now], sort = True)
            
        structure_now = metadata.iloc[cell_id, :]["structure"]
        met_type_now = metadata.iloc[cell_id, :]["met_type"]
        t_type_now = metadata.iloc[cell_id, :]["Unnamed: 21"]
        
        specimen_id.append(specimen_id_now)
        structure.append(structure_now)
        met_type.append(met_type_now)
        t_type.append(t_type_now)
        V.append(best_sweep)

        print("%4.f Specimen ID: %d | %s | %s | Sweep ID: %d | Init. freq.: %.2f Hz" % 
              (cell_id, specimen_id_now, structure_now, met_type_now, best_sweep_id, best_init_freq))

   0 Specimen ID: 601506507 | VISpm4 | Vip-MET-2 | Sweep ID: 19 | Init. freq.: 98.81 Hz


   1 Specimen ID: 601803754 | VISp2/3 | Sst-MET-3 | Sweep ID: 7 | Init. freq.: 84.60 Hz
   2 Specimen ID: 601808698 | VISp5 | Sst-MET-8 | Sweep ID: 20 | Init. freq.: 106.38 Hz
   3 Specimen ID: 601810307 | VISp2/3 | Sst-MET-3 | Sweep ID: 31 | Init. freq.: 104.60 Hz
   4 Specimen ID: 601811091 | VISp6a | Sst-MET-13 | Sweep ID: 23 | Init. freq.: 84.89 Hz
   5 Specimen ID: 602231708 | VISp5 | Sst-MET-4 | Sweep ID: 20 | Init. freq.: 107.76 Hz
   6 Specimen ID: 602535278 | VISp2/3 | Sst-MET-3 | Sweep ID: 27 | Init. freq.: 102.25 Hz
   7 Specimen ID: 604646725 | VISp2/3 | Vip-MET-1 | Sweep ID: 21 | Init. freq.: 90.58 Hz
   8 Specimen ID: 605060256 | VISal4 | Vip-MET-4 | Sweep ID: 29 | Init. freq.: 69.16 Hz
   9 Specimen ID: 605160536 | VISp2/3 | Vip-MET-5 | Sweep ID: 5 | Init. freq.: 105.26 Hz
  10 Specimen ID: 606098116 | VISp2/3 | Pvalb-MET-4 | Sweep ID: 16 | Init. freq.: 51.65 Hz
  11 Specimen ID: 606271263 | VISp2/3 | Pvalb-MET-5 | Sweep ID: 26 | Init. freq.: 108.70 Hz
  12 Specimen ID: 

  16 Specimen ID: 623434306 | VISp2/3 | Sst-MET-2 | Sweep ID: 24 | Init. freq.: 99.01 Hz
  17 Specimen ID: 626153003 | VISp5 | Sst-MET-12 | Sweep ID: 26 | Init. freq.: 108.23 Hz


  18 Specimen ID: 627881581 | VISp4 | Sst-MET-8 | Sweep ID: 27 | Init. freq.: 105.93 Hz


  19 Specimen ID: 637846989 | VISp2/3 | Lamp5-MET-1 | Sweep ID: 33 | Init. freq.: 55.49 Hz
  20 Specimen ID: 639004612 | VISpm2/3 | Vip-MET-1 | Sweep ID: 17 | Init. freq.: 103.73 Hz


...

 490 Specimen ID: 992268582 | VISp2/3 | Pvalb-MET-5 | Sweep ID: 11 | Init. freq.: 94.88 Hz


 491 Specimen ID: 992386952 | VISl1 | Sncg-MET-1 | Sweep ID: 8 | Init. freq.: 94.88 Hz


 492 Specimen ID: 992830261 | VISp5 | Sncg-MET-3 | Sweep ID: 10 | Init. freq.: 45.17 Hz


 493 Specimen ID: 993245688 | VISp2/3 | Sncg-MET-1 | Sweep ID: 20 | Init. freq.: 47.76 Hz


 494 Specimen ID: 993283588 | VISp2/3 | Sncg-MET-1 | Sweep ID: 17 | Init. freq.: 80.78 Hz


In [7]:
all_ephys_features.to_csv(save_path + 'v1_patchseq_ephys_features.csv') 

In [8]:
V = np.vstack(V)
V.shape

(494, 21000)

In [9]:
with open(save_path + 'voltage_traces.pickle', 'wb') as f:
    pickle.dump({"V": V, "specimen_id": specimen_id, "structure": structure, 
                 "met_type": met_type, "t_type": t_type}, f)